In [1]:
import requests
from urllib.parse import quote
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

### 1. 대상지 주소 입력 > 법정동코드 추출

대상지 : 강원 철원군 갈말읍 군탄리 473-1 ( DMZ일원 화살머리고지 등 유해발굴지 )

In [2]:
from urllib.parse import quote
with open('road_key.txt') as file:
    road_key = f'{file.read()}'

In [3]:
road_url = 'https://www.juso.go.kr/addrlink/addrLinkApi.do'
params1 = f'confmKey={road_key}&currentPage=1&countPerPage=10'
params2 = f'keyword={quote(input())}&resultType=json'
url = f"{road_url}?{params1}&{params2}"
result = requests.get(url).json()

BJD_code=result['results']['juso'][0]['admCd'] 
BJD_code

'5013025926'

### 2. 법정동코드 > 대상지 기후대, 토양성분 추출

In [4]:
#https://www.forest.go.kr/kfsweb/kfi/kfs/trail/soil.do?mn=NKFS_06_08_02&fileDownload=Y&dataType=/soil/FS_IJ100_50.zip&pblicDataId=PBD0000181&tabs=1&searchSrvc=&subTitle=&searchWrd=&searchCnd=
addrr = pd.read_csv('전국.csv', sep=',')  #전국.csv만들기
addrr = pd.DataFrame(addrr)
addrr = addrr[['EMNDN_CD','CLZN_CD']]
addrr.columns = ['읍면동코드','기후대코드']
addrr['읍면동코드'] = int(BJD_code[:-2])
climate_code = addrr['기후대코드'].mode()[0]
climate_code  

# 1) 온대북부, 2)온대중부, 3)온대남부 4) 난대

# if climate_code == 1.0:
#     print('온대북부')
# elif climate_code == 2.0:
#     print('온대중부')
# elif climate_code == 3.0:
#     print('온대남부')
# elif climate_code == 4.0:
#     print('난대')

C:\Users\ICT01_11\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


2.0

In [5]:
with open('soil_key.txt') as file:
    soilkey = file.read()

s_url = f'http://apis.data.go.kr/1390802/SoilEnviron/SoilExam/getSoilExamList?serviceKey={soilkey}&Page_Size=10&Page_No=1&BJD_Code={BJD_code}'
s_result = requests.get(s_url)
s_contents = s_result.text
s_soup = BeautifulSoup(s_contents, 'html.parser')

acid = []
vldpha = []
vldsia = []
om = []
posifert_mg = []
posifert_k = []
posifert_ca = []

lis = [acid, vldpha, vldsia, om, posifert_mg, posifert_k, posifert_ca]
cons = ['acid', 'vldpha', 'vldsia', 'om', 'posifert_mg', 'posifert_k', 'posifert_ca']

for li in lis:
    for con in cons:
        if lis.index(li) == cons.index(con):
            for a in s_soup.find_all(con):
                li.append(float(a.get_text()))

ground_con = pd.DataFrame({'산도':acid, '유효인산':vldpha, '유효규산':vldsia, '유기물':om, '마그네슘':posifert_mg, '칼륨':posifert_k , '칼슘':posifert_ca})
ground_con

,산도,유효인산,유효규산,유기물,마그네슘,칼륨,칼슘
0,4.5,60,,159,.4,.56,.1
1,5.1,41,,125,1.1,.88,3.4
2,5.7,19,,182,1.3,.79,2.6
3,5.9,50,,197,1.4,.63,3.6
4,6.4,33,,123,4.1,.6,9.1
5,6.1,127,,162,2.1,.89,3.3
6,5.6,51,,145,.6,.44,1.5
7,6,14,,116,.6,.57,1.4
8,5.9,10,,94,.8,.61,1.9
9,5.6,45,,148,1,.58,1.5


- 토양 유형 api << 쓰지 말자!! >>

In [6]:
import requests
from urllib.parse import quote
import pandas as pd

In [7]:
with open('soil_type_key.txt') as file:
    type_key = file.read()

In [8]:
url = f'http://apis.data.go.kr/1390802/SoilEnviron/SoilCharac2/getSoilCharacter2?serviceKey={type_key}&PNU_Code=4215034022100050000'
url

'http://apis.data.go.kr/1390802/SoilEnviron/SoilCharac2/getSoilCharacter2?serviceKey=MAW83St0u7MX4NvrMdB3L815EvMa%2Bl5IQE2mS8jMIHr49AXVHFmUbSzGp%2FqZEk3C2ckB5UcIgajRm8%2BreDnYEg%3D%3D&PNU_Code=4215034022100050000'

In [9]:
result = requests.get(url)
content = result.text

In [10]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(content, 'html.parser')       # parser: 구문분석기, 원하는 형태로 만들어주기 
print(soup)


<response>
<header>
<result_code>200</result_code>
<result_msg>OK</result_msg>
</header>
<body>
<items>
<item>
<pnu_code>4215034022100050000</pnu_code>
<soil_color_code>08</soil_color_code>
<deepsoil_qlt_code>04</deepsoil_qlt_code>
<soil_structure_code>04</soil_structure_code>
<deepsoil_color_code>01</deepsoil_color_code>
<deepsoil_ston_code>02</deepsoil_ston_code> <soilslope_code>03</soilslope_code>
<matrix_code>04</matrix_code>
<soildra_code>02</soildra_code>
<vldsoildep_code>03</vldsoildep_code>
<erosion_code>01</erosion_code>
<surtture_code>04</surtture_code>
<sur_ston_code>02</sur_ston_code>
<soil_type_geo_code>04</soil_type_geo_code>
<accu_style_code>03</accu_style_code>
<main_order_code>06</main_order_code>
<sub_order_code>12</sub_order_code>
<grategroup_code>26</grategroup_code>
<main_landuse_code>02</main_landuse_code>
<soil_use_rec_code>02</soil_use_rec_code>
<soil_type_code>07</soil_type_code>
<rfld_grd_code>04</rfld_grd_code>
<paddy_factor_code>02</paddy_factor_code>
<pfld

In [11]:
soups = soup.find('soil_type_code')
print(soups.string)
 
# 07 : 밭, 보통전 

07


In [12]:
# 지번코드 -> 
# 자생식물 주소-> 지번코드 
# 